In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/hyejin/Downloads/my-semi-structure-recsys-67625-3fa813796d71.json"



In [2]:
PROJECT_ID = "my-semi-structure-recsys-67625"
REGION = 'US'

In [4]:
# !bq mk --location=$REGION --dataset $PROJECT_ID:ecommerce

In [3]:
import pandas_gbq

# TODO: Set project_id to your Google Cloud Platform project ID.
# project_id = "my-project"

query = """
SELECT 
  CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId,
  hitNumber,
  time,
  page.pageTitle,
  type,
  productSKU,
  v2ProductName,
  v2ProductCategory,
  productPrice/1000000 as productPrice_USD

FROM 
  `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, 
  UNNEST(hits) AS hits,
  UNNEST(hits.product) AS hits_product
LIMIT 5
"""
df = pandas_gbq.read_gbq(query, project_id=PROJECT_ID)
df.head()

/usr/local/lib/python3.8/site-packages/google/cloud/bigquery/table.py:1923: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,visitorId,hitNumber,time,pageTitle,type,productSKU,v2ProductName,v2ProductCategory,productPrice_USD
0,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056299,Gift Card - $25.00,Home/Gift Cards/,25.00
1,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056499,Gift Card - $50.00,Home/Gift Cards/,50.00
2,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056199,Gift Card- $100.00,Home/Gift Cards/,100.00
3,7194065619159478122-1,2,30507,Gift Cards,PAGE,GGOEGGCX056399,Gift Card - $250.00,Home/Gift Cards/,250.00
4,8159312408158297118-151,2,289206,Men's-T-Shirts,PAGE,GGOEGAAX0104,Google Men's 100% Cotton Short Sleeve Hero Tee...,Home/Apparel/Men's/Men's-T-Shirts/,16.99


In [ ]:
# WITH product AS (
#   SELECT 
#       productSKU AS id,
#       v2ProductName AS title,
#       'USD' AS currency_code,
#       ARRAY_AGG(DISTINCT(split(v2ProductCategory,'/')[OFFSET(1)])) AS categories,
#       max(productPrice/1000000) as original_price,
#       min(productPrice/1000000) as display_price
#     FROM 
#       `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, 
#       UNNEST(hits) AS hits,
#       UNNEST(hits.product) AS hits_product
#     WHERE v2ProductCategory like "%/%" 
#     GROUP BY 1,2,3
# ), aggregate AS (
#     SELECT id, 
#       currency_code,
#       ARRAY_AGG(STRUCT(original_price, display_price)) AS price
#     FROM product 
#     GROUP BY 1,2
#   ), meta AS (
#     SELECT id,
#       ARRAY_AGG(STRUCT(price, currency_code)) AS product_metadata
#   FROM aggregate 
#   GROUP BY 1
#   ),hierarchies AS (
#     SELECT id,
#       ARRAY_AGG(STRUCT(categories)) AS category_hierarchies
#   FROM product 
#   GROUP BY 1
#   )
#   SELECT product.id,
#       title,
#       category_hierarchies,
#       product_metadata
#   FROM product INNER JOIN meta ON product.id = meta.id INNER JOIN hierarchies ON product.id = hierarchies.id 


In [87]:
from google.cloud import bigquery
client = bigquery.Client()
query = """
CREATE OR REPLACE TABLE  `my-semi-structure-recsys-67625.ecommerce.product_catalog` AS (
  WITH product AS (
  SELECT 
      productSKU AS id,
      v2ProductName AS title,
      'USD' AS currency_code,
      ARRAY_AGG(DISTINCT(split(v2ProductCategory,'/')[OFFSET(1)])) AS categories
    FROM 
      `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`, 
      UNNEST(hits) AS hits,
      UNNEST(hits.product) AS hits_product
    WHERE v2ProductCategory like "%/%" 
    GROUP BY 1,2,3
),hierarchies AS (
    SELECT id,
      ARRAY_AGG(STRUCT(categories)) AS category_hierarchies
  FROM product 
  GROUP BY 1
  )
  SELECT product.id,
      title,
      category_hierarchies
  FROM product INNER JOIN hierarchies ON product.id = hierarchies.id 
);
SELECT * 
FROM `my-semi-structure-recsys-67625.ecommerce.product_catalog`
"""
query_job = client.query(query)
df = query_job.result().to_dataframe()

df.head()

,id,title,category_hierarchies
0,GGOEGAAX0296,YouTube Women's Short Sleeve Tri-blend Badge T...,"[{'categories': ['Brands', 'Apparel', 'Shop by..."
1,GGOEGAAX0300,YouTube Women's Racer Back Tank Black,"[{'categories': ['Brands', 'Apparel', 'Shop by..."
2,GGOEGAAX0630,Android Baby Essentials Baby Set,"[{'categories': ['Brands', 'Apparel', 'Shop by..."
3,GGOEGAAX0688,Android Youth Short Sleeve T-shirt Pewter,"[{'categories': ['Brands', 'Apparel', 'Shop by..."
4,GGOEGAAX0687,Android Youth Short Sleeve T-shirt Aqua,"[{'categories': ['Brands', 'Apparel', 'Shop by..."


In [86]:
df.product_metadata[0]

array([{'price': array([{'original_price': 2.5, 'display_price': 2.0}], dtype=object), 'currency_code': 'USD'}],
      dtype=object)

In [97]:
import json
df.to_json('product_catalog.json', orient="records")


In [98]:
!cat product_catalog.json | jq -c '.[]' > catalog.json

In [101]:
!gsutil cp catalog.json gs://semi-structure-67625-bucket/ecommerce/catalog.json


Copying file://catalog.json [Content-Type=application/json]...

Operation completed over 1 objects/28.8 KiB.                                     


In [103]:
# !bq show
!bq load --replace --source_format=NEWLINE_DELIMITED_JSON ecommerce.product_catalog gs://semi-structure-67625-bucket/ecommerce/catalog.json catalog_schema.json



Waiting on bqjob_r113da65bbb25a1b3_0000017af4ba5f82_1 ... (5s) Current status: DONE   


1. dataframe -> json변환 
df.to_json([before.json], orient="records")

2. json -> newline_delimited_format json 변환
!cat [before.json] | jq -c '.[]' > [after.json]

3. json을 google storage bucket에 업로드
!gsutil cp catalog.json gs://semi-structure-67625-bucket/[ 버킷 디렉토리 ]

4. gs bucket에서 빅쿼리로 정의한 스키마 파일과 함께 업로드
!bq load --replace --source_format=NEWLINE_DELIMITED_JSON [빅쿼리 테이블] gs://semi-structure-67625-bucket/[ 버킷 디렉토리 ] 스키마.json

5. 빅쿼리에서 recommendation ai로 import

Required for initial live experiment : add-to-cart, detail-page-view, home-page-view, purchase-complete

In [14]:
from google.cloud import bigquery
client = bigquery.Client()
query = """
CREATE OR REPLACE TABLE  `my-semi-structure-recsys-67625.ecommerce.user_events_detail_page_view` AS (
  WITH detail_page_view AS (
  SELECT 
      "detail-page-view" AS eventType,
      STRING(TIMESTAMP_SECONDS(SAFE_CAST(visitStartTime+hits.time/1000 AS INT64)), 'UTC') as eventTime,
      productSKU AS id,
      CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId      
    FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_2016*` t,
      UNNEST(hits) AS hits,
      UNNEST(hits.product) AS hits_product
    WHERE
     eCommerceAction.action_type = "2" 
  ),
  product_detail AS (
    SELECT eventType,
      eventTime,
      ARRAY_AGG(STRUCT(id)) AS productDetails
    FROM detail_page_view
    GROUP BY 1,2
  ),
  product_event_detail AS (
    SELECT eventType,
      eventTime,
      ARRAY_AGG(STRUCT(productDetails)) AS productEventDetail
    FROM product_detail
    GROUP BY 1,2
  ),
  user_info AS (
    SELECT eventType,
      eventTime,
      ARRAY_AGG(STRUCT(visitorId)) AS userInfo
    FROM detail_page_view 
    GROUP BY 1,2
  )
    SELECT detail_page_view.eventType,
      detail_page_view.eventTime,
      productEventDetail,
      userInfo
    FROM detail_page_view INNER JOIN product_event_detail ON detail_page_view.eventTime = product_event_detail.eventTime
        INNER JOIN user_info ON detail_page_view.eventTime = user_info.eventTime
    );
SELECT * 
FROM `my-semi-structure-recsys-67625.ecommerce.user_events_detail_page_view`
"""
query_job = client.query(query)
df = query_job.result().to_dataframe()

df.head()

,eventType,eventTime,productEventDetail,userInfo
0,detail-page-view,2016-10-21 03:14:40+00,[{'productDetails': [{'id': 'A12345'}]}],[{'visitorId': '9006029694274259015-3'}]
1,detail-page-view,2016-10-12 18:19:03+00,[{'productDetails': [{'id': 'GGOEGAAX0031'}]}],[{'visitorId': '4797728069033938331-1'}]
2,detail-page-view,2016-10-25 06:39:33+00,[{'productDetails': [{'id': 'GGOEGAAX0031'}]}],[{'visitorId': '229095768299079389-3'}]
3,detail-page-view,2016-12-12 04:42:00+00,[{'productDetails': [{'id': 'GGOEGAAX0031'}]}],[{'visitorId': '1928170213555367320-3'}]
4,detail-page-view,2016-09-15 23:14:16+00,[{'productDetails': [{'id': 'GGOEGAAX0031'}]}],[{'visitorId': '6535875617262565784-2'}]


In [15]:
import json
df.to_json('detail_page_view.json', orient="records")


In [16]:
!cat detail_page_view.json | jq -c '.[]' > user_events_detail_page_view.json

In [98]:
!gsutil cp user_events_detail_page_view.json gs://semi-structure-67625-bucket/ecommerce/detail_page_view.json


Copying file://user_events_detail_page_view.json [Content-Type=application/json]...

Operation completed over 1 objects/30.8 MiB.                                     


In [99]:
# !bq show
!bq load --replace --source_format=NEWLINE_DELIMITED_JSON ecommerce.detail_page_view gs://semi-structure-67625-bucket/ecommerce/detail_page_view.json detail_page_view_schema.json



Waiting on bqjob_r7afe079adc766521_0000017af6923876_1 ... (17s) Current status: DONE   


In [103]:
from google.cloud import bigquery
client = bigquery.Client()
query = """
CREATE OR REPLACE TABLE  `my-semi-structure-recsys-67625.ecommerce.detail_page_view_temp` AS (
  WITH page_view_time AS (
    SELECT DISTINCT 
      "detail-page-view" AS eventType,
      DATETIME_ADD(DATETIME(FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TIMESTAMP_SECONDS(SAFE_CAST(visitStartTime+hits.time/1000 AS INT64)),'UTC')), INTERVAL 4 YEAR) AS eventTime,
      productSKU as id,
      CONCAT(fullVisitorID,'-',CAST(visitNumber AS STRING)) AS visitorId
    FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_2016*` t,
      UNNEST(hits) AS hits,
      UNNEST(hits.product) AS hits_product
    WHERE
      eCommerceAction.action_type = "2"
), details AS (
    SELECT eventType, DATETIME_ADD(eventTime, INTERVAL 7 MONTH) AS eventTime, visitorId, ARRAY_AGG(STRUCT(id)) AS productDetails FROM page_view_time
    GROUP BY 1,2,3
    ORDER BY 1,2,3
)
   SELECT eventType, CONCAT(EXTRACT(DATE FROM eventTime), 'T', EXTRACT(TIME FROM eventTime), 'Z') AS eventTime, ARRAY_AGG(STRUCT(visitorId)) AS userInfo, ARRAY_AGG(STRUCT(productDetails)) AS productEventDetail FROM details
    GROUP BY 1,2
    ORDER BY 1,2
);
SELECT * 
FROM `my-semi-structure-recsys-67625.ecommerce.detail_page_view_temp`
"""
query_job = client.query(query)
df = query_job.result().to_dataframe()

df.head()

,eventType,eventTime,userInfo,productEventDetail
0,detail-page-view,2021-03-01T07:16:57Z,[{'visitorId': '4283942788674999975-1'}],[{'productDetails': [{'id': 'GGOEGEDQ023899'}]}]
1,detail-page-view,2021-03-01T07:19:10Z,[{'visitorId': '4283942788674999975-1'}],[{'productDetails': [{'id': 'GGOEYDHJ056099'}]}]
2,detail-page-view,2021-03-01T07:19:11Z,[{'visitorId': '4283942788674999975-1'}],[{'productDetails': [{'id': 'GGOEYDHJ056099'}]}]
3,detail-page-view,2021-03-01T07:19:24Z,[{'visitorId': '4283942788674999975-1'}],[{'productDetails': [{'id': 'GGOEYDHJ019399'}]}]
4,detail-page-view,2021-03-01T07:19:40Z,[{'visitorId': '4283942788674999975-1'}],[{'productDetails': [{'id': 'GGOEGDHC017999'}]}]


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175494 entries, 0 to 175493
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   eventType           175494 non-null  object
 1   eventTime           175494 non-null  object
 2   userInfo            175494 non-null  object
 3   productEventDetail  175494 non-null  object
dtypes: object(4)
memory usage: 5.4+ MB


In [123]:
import json
df.to_json('detail_page_view.json', orient="records")


In [105]:
!cat detail_page_view.json | jq -c '.[]' > user_events_detail_page_view.json

In [129]:
with open('user_events_detail_page_view.json', 'r') as file:
    data = file.read().replace('userInfo":[', 'userInfo":')
    data = data.replace('}],', '},')
    data = data.replace('productEventDetail":[', 'productEventDetail":')
    data = data.replace('"}]}]', ':}]}')
file = open('user_events_detail_page_view.json','w')
file.write(data)

32086914

In [96]:
!gsutil cp user_events_detail_page_view.json gs://semi-structure-67625-bucket/ecommerce/detail_page_view.json

Copying file://user_events_detail_page_view.json [Content-Type=application/json]...

Operation completed over 1 objects/30.8 MiB.                                     


In [97]:
# !bq show
!bq load --replace --source_format=NEWLINE_DELIMITED_JSON ecommerce.detail_page_view gs://semi-structure-67625-bucket/ecommerce/detail_page_view.json detail_page_view_schema.json



Waiting on bqjob_rf1626c690e28be1_0000017af68a81d1_1 ... (17s) Current status: DONE   
